### Sample Syntax for P0 Map

#### Usman Asad, Faisal Jaffer, Aleem Haq. Modified in April 2021

This notebooks contains sample syntax and sample execution of maps in P0

In [1]:
import time 
import nbimporter; nbimporter.options["only_defs"] = False
from P0 import compileString
from ST import printSymTab

def runpywasm(wasmfile):
    import pywasm
    def write(s, i): print(i)
    def writeln(s): print('\n')
    def read(s): return int(input())
    def time_(s): 
        print(time.time() * 1000)
        return time.time() * 1000
    def key_err(s):
        raise Exception("Key does not exist")
    vm = pywasm.load(wasmfile, {'P0lib': {'write': write, 'writeln': writeln, 'read': read, 'time': time_,'err': key_err}})
    
def runwasm(wasmfile):
    from IPython.core.display import display, Javascript
    display(Javascript("""
    const params = { 
        P0lib: { 
            write: i => this.append_stream({text: '' + i, name: 'stdout'}),
            writeln: () => this.append_stream({text: '\\n', name: 'stdout'}),
            read: () => window.prompt()
        }
    }

    fetch('""" + wasmfile + """') // asynchronously fetch file, return Response object
      .then(response => response.arrayBuffer()) // read the response to completion and stores it in an ArrayBuffer
      .then(code => WebAssembly.compile(code)) // compile (sharable) code.wasm
      .then(module => WebAssembly.instantiate(module, params)) // create an instance with memory
    // .then(instance => instance.exports.program()); // run the main program; not needed if start function specified
    """))
    
from wasmer import engine, Store, Module, Instance, ImportObject, Function
from wasmer_compiler_cranelift import Compiler

def runwasmer(wasmfile):
    def write(i: int): print(i)
    def writeln(): print('\n')
    def read() -> int: return int(input()) 
    store = Store(engine.JIT(Compiler))
    module = Module(store, open(wasmfile, 'rb').read())
    import_object = ImportObject()
    import_object.register("P0lib", {"write": Function(store, write),
                                     "writeln": Function(store, writeln),"read": Function(store, read)})
    instance = Instance(module, import_object)

Importing Jupyter notebook from P0.ipynb
Importing Jupyter notebook from SC.ipynb
Importing Jupyter notebook from ST.ipynb


# Syntax

### Map declaration 
`var example: map <type₁, type₂>` where type₁ & type₂ can be any 32 bit word, and example can be any variable name that fits within the P0 compiler.



#### The map `example` defined above will be used as an example for the following definitions
### Use of Functioons
The way to use the Map's functions is to add a `.` after the variable name, followed by a command. e.g. `example.init`


### Initialization    
`example.init` initializes the map. This needs to be done once after declaration in order for you to use the map that you defined

### Insertion 
`example.insert(key,value)` is used to insert a value into the map. The key and value have to be of types; `type₁`, and `type₂` respectively. If not an exception is thrown.

### Get
`example.get(key)` is used to get the value of the key given. If the key does not exist an error is thrown
### Length
`example.len()` is used to return the number of entries in the map. 

<b>Map example with key and value as integer type</b>

In [2]:
compileString("""
program bitsets
  var c: integer
  var e: map <integer, integer>
    c := 4
    e.init
    e.insert(1,10)
    e.insert(2,9)
    e.insert(2, 4)
    e.insert(3, 4)
    e.insert(4, 4)
    e.insert(5, 4)
    e.insert(6, 4)
    e.insert(7, 4)
    e.insert(8, 4)
    e.insert(9, 4)
    e.insert(10, 4)
    e.insert(13, 4)
    e.insert(14, 4)
    e.insert(15, 4)
    e.insert(16, 4)
    e.insert(17, 4)
    e.insert(18, 4)
    e.insert(19, 4)
    e.insert(20, 4)
    write(e.get(2))
    write(e.get(3))
    write(e.get(20))
""", 'primes.wat', target = 'wat')
!wat2wasm primes.wat

Importing Jupyter notebook from CGwat.ipynb
Importing Jupyter notebook from Map_Proto.ipynb


In [3]:
runpywasm("primes.wasm")

4
4
4


<b>Map example with key as integer type and value as boolean</b>

In [4]:
compileString("""
program bitsets
  var a, b: set [1 .. 5]
  var z: [1 .. 5] → integer
  var y: boolean
  var c: integer
  var e: map <integer, boolean>
  var d: boolean
    e.init
    c := 2
    e.insert(c,true)
    e.insert(1, false)
    write(e.get(c))
    write(e.get(1))
""", 'primes.wat', target = 'wat')
!wat2wasm primes.wat

In [5]:
runpywasm("primes.wasm")

1
0


<b>Map example with for key in map</b>

In [6]:
compileString("""
program bitsets
  var a, b: set [1 .. 5]
  var z: [1 .. 5] → integer
  var y: boolean
  var c: integer
  var e: map <integer, boolean>
  var d: boolean
    e.init
    c := 2
    e.insert(c,true)
    e.insert(1, false)
    if 3 in e then write(1)
    else write(0)
    if c in e then write(1)
    else write(0)
    if 1 in e then write(1)
    else write(0)
""", 'primes.wat', target = 'wat')
!wat2wasm primes.wat

In [7]:
runpywasm("primes.wasm")

0
1
1
